In [2]:
import os
if not 'juxtorpus' in os.listdir():
    os.chdir('../')
assert 'juxtorpus' in os.listdir(), f"Working directory should be at juxtorpus. But at {os.getcwd()}"
f"Working directory: {os.getcwd()}"

'Working directory: /Users/hcha9747/workspace/juxtorpus'

# File Upload Widget

In [2]:
from juxtorpus.viz.widgets import FileUploadWidget

uploader = FileUploadWidget(['.csv', '.zip'])
uploader.render()
# If your 'upload' operation is exactly the same, the widget will automatically do nothing.
# i.e. 
# if you upload fileA and then fileA again. The second upload does nothing.
# if you upload fileA, fileB and then just fileA. The second upload is accepted. fileA however will be rejected but this is not part of the underlying ipywidget behaviour.

In [ ]:
uploader.uploaded()

# File Checks

In [10]:
from juxtorpus.checks import FileCheckers, FileSizeCheck, EncodingCheck

checks = [
    FileSizeCheck(max_bytes=20*1<<20),   # 10 megabytes
    EncodingCheck(expected='utf-8', min_rows_to_check=100)
]
file_checks = FileCheckers(checks)

In [11]:
import pathlib
flagged = file_checks.run(uploader.uploaded())
flagged

{}

In [12]:
file_checks.summary()

Flagged    0
Passed     1
Total      1
Name: File Check Summary, dtype: UInt16

In [13]:
file_checks.passed(), file_checks.flagged()

(['/var/folders/jh/mqv5k4x56d98m2twhmk4bjdc0000gp/T/tmpix77s9q4/Geolocated_places_climate_with_LGA_and_remoteness_with_text.csv'],
 {})

# Building the Corpus

In [14]:
from juxtorpus.corpus import CorpusBuilder

builder = CorpusBuilder(file_checks.passed())
builder.show_columns()

,All Columns,Added
0,year,
1,month,
2,day,
3,lat_mid,
4,lon_mid,
5,screen_name,
6,tweet_id,
7,retweet,
8,text,
9,geometry,


In [15]:
builder.set_corpus_type('corpus')
builder.set_text_column('doc')
builder.set_nrows(1000)
builder.set_sep(',')
builder.add_meta('created_at', dtype='datetime', lazy=True)
builder.add_meta('id', dtype='int', lazy=False)
builder.show_columns()

ValueError: created_at column does not exist.

In [10]:
corpus = builder.build()

In [11]:
corpus.metas()

frozendict.frozendict({'created_at': <SeriesMeta [Id: created_at]>, 'id': <SeriesMeta [Id: id]>})

In [12]:
corpus.metas().get('created_at').preview(5)

0   2017-04-05 14:56:29
1   2017-04-03 20:01:01
2   2017-04-03 16:59:35
3   2017-04-03 16:30:19
4   2017-04-02 06:05:23
Name: created_at, dtype: datetime64[ns]

In [13]:
from juxtorpus.corpus import CorpusSlicer
import pandas as pd

slicer = CorpusSlicer(corpus)
corpus_subset = slicer.filter_by_condition('created_at', lambda x: x > pd.Timestamp('2017-04-02'))

"Original: {} documents.".format(len(corpus)), "Slice: {} documents".format(len(corpus_subset))

('Original: 1000 documents.', 'Slice: 16 documents')

In [14]:
corpus_subset.metas().get('id').preview(5)

0    849636868052275200
1    848988730585096192
2    848943072423497728
3    848935705057280001
4    848416049573658624
Name: id, dtype: int64

In [15]:
another_meta = corpus_subset.metas().get('id').apply(lambda x: x*-1)
another_meta

0   -849636868052275200
1   -848988730585096192
2   -848943072423497728
3   -848935705057280001
4   -848416049573658624
5   -848415731502923777
6   -848415356263702528
7   -848398971139629057
0   -849636868052275200
1   -848988730585096192
2   -848943072423497728
3   -848935705057280001
4   -848416049573658624
5   -848415731502923777
6   -848415356263702528
7   -848398971139629057
Name: id, dtype: int64

In [16]:
from juxtorpus.meta import SeriesMeta

series_meta = SeriesMeta(id_='anothermeta', series=another_meta)

corpus_subset.add_meta(series_meta)

corpus_subset.metas()

frozendict.frozendict({'created_at': <SeriesMeta [Id: created_at]>, 'id': <SeriesMeta [Id: id]>, 'anothermeta': <SeriesMeta [Id: anothermeta]>})

In [17]:
slicer = CorpusSlicer(corpus_subset)
corpus_subset_subset = slicer.filter_by_condition('anothermeta', lambda x: x > 0)

In [19]:
len(corpus), len(corpus_subset), len(corpus_subset_subset)

(1000, 16, 0)

In [20]:
corpus.metas(), corpus_subset.metas(), corpus_subset_subset.metas()

(frozendict.frozendict({'created_at': <SeriesMeta [Id: created_at]>, 'id': <SeriesMeta [Id: id]>}),
 frozendict.frozendict({'created_at': <SeriesMeta [Id: created_at]>, 'id': <SeriesMeta [Id: id]>, 'anothermeta': <SeriesMeta [Id: anothermeta]>}),
 frozendict.frozendict({'created_at': <SeriesMeta [Id: created_at]>, 'id': <SeriesMeta [Id: id]>, 'anothermeta': <SeriesMeta [Id: anothermeta]>}))